In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import os

# to make this notebook's output stable across runs
np.random.seed(42)
tf.random.set_seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

2023-11-10 14:58:28.110037: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-10 14:58:28.211933: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [2]:
fashion_mnist = keras.datasets.fashion_mnist
(X_train_full, y_train_full), (X_test, y_test) = fashion_mnist.load_data()

X_valid, X_train = X_train_full[:5000] / 255., X_train_full[5000:] / 255.
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]
X_test = X_test / 255.

In [3]:
class_names = ["T-shirt/top", "Trouser", "Pullover", "Dress", "Coat",
               "Sandal", "Shirt", "Sneaker", "Bag", "Ankle boot"]

In [4]:
def build_model (n_hidden=3, n_units=100, learning_rate=0.01):
    model = keras.models.Sequential()
    model.add(keras.layers.Flatten(input_shape=[28, 28]))
    for i in range(n_hidden):
        model.add(keras.layers.Dense(n_units, activation="relu"))
    model.add(keras.layers.Dense(10, activation="softmax"))

    model.compile(loss="sparse_categorical_crossentropy",
              optimizer=keras.optimizers.SGD(lr=learning_rate),
              metrics=["accuracy"])
    return model

In [6]:
from scipy.stats import reciprocal
import sklearn
from sklearn.model_selection import RandomizedSearchCV

param_distribs = {
    "n_hidden": [0, 1, 2, 3],
    "n_units": np.arange(1, 100).tolist(),
    "learning_rate": reciprocal(3e-4, 3e-2).rvs(1000).tolist(),
}

# wrapper
from scikeras.wrappers import KerasClassifier

keras_clf = KerasClassifier(
    model=build_model,
    loss="sparse_categorical_crossentropy",
    n_units=1, n_hidden=1, learning_rate=0.001 # necessary to assign a value here, to make them visible (and tunable) to scikit-learn
)

rnd_search_cv = RandomizedSearchCV(keras_clf, param_distribs, n_iter=7, cv=2, verbose=2) # toy params
rnd_search_cv.fit(X_train, y_train, epochs=45,
                  validation_data=(X_valid, y_valid),
                  callbacks=[keras.callbacks.EarlyStopping(patience=5)])

2023-11-10 15:06:22.917485: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Fitting 2 folds for each of 7 candidates, totalling 14 fits
Epoch 1/45
860/860 [==============================] - 1s 720us/step - loss: 0.9426 - accuracy: 0.7079 - val_loss: 0.6429 - val_accuracy: 0.7984
Epoch 2/45
860/860 [==============================] - 1s 616us/step - loss: 0.6025 - accuracy: 0.8022 - val_loss: 0.5532 - val_accuracy: 0.8150
Epoch 3/45
860/860 [==============================] - 1s 588us/step - loss: 0.5405 - accuracy: 0.8197 - val_loss: 0.5248 - val_accuracy: 0.8192
Epoch 4/45
860/860 [==============================] - 1s 592us/step - loss: 0.5080 - accuracy: 0.8272 - val_loss: 0.4946 - val_accuracy: 0.8342
Epoch 5/45
860/860 [==============================] - 1s 594us/step - loss: 0.4880 - accuracy: 0.8337 - val_loss: 0.5078 - val_accuracy: 0.8174
Epoch 6/45
860/860 [==============================] - 1s 598us/step - loss: 0.4719 - accuracy: 0.8371 - val_loss: 0.4582 - val_accuracy: 0.8466
Epoch 7/45
860/860 [==============================] - 1s 608us/step - loss: 

KeyboardInterrupt: 

In [31]:
rnd_search_cv.best_params_

{'n_units': 90, 'n_hidden': 3, 'learning_rate': 0.001774190610836516}

In [33]:
model = rnd_search_cv.best_estimator_.model
model.evaluate(X_test, y_test)

313/313 [==============================] - 0s 526us/step - loss: 0.3377 - accuracy: 0.8783


[0.3376643657684326, 0.8783000111579895]